# Stage 0: Crafting Effective System Prompts

<a href="https://colab.research.google.com/github/redislabs-training/ce-redis-langchain/blob/main/section-1-context-engineering-foundations/03_stage_0_crafting_effective_system_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Welcome to Stage 0 of the course. Before we build complex retrieval systems, we need to define who our agent is. In this preliminary stage, we'll focus on the foundation of all context: **System Instructions**.

More specifically, we'll do the following:

1.  **Define Identity**: Learn how to set the agent's role, capabilities, and constraints.
2.  **Experiment with Prompts**: See firsthand how different system instructions change the agent's behavior.
3.  **Apply to the Agent**: Use a special hook to inject your custom system prompt into our Stage 1 Baseline Agent.

Let's dive in.

## Concepts: System Instructions

### What Are System Instructions?

System instructions (also called system prompts) are the **persistent context** that defines your agent's identity and behavior. They are included in every conversation turn and tell the LLM:

1.  **Who it is** - Role and identity
2.  **What it can do** - Capabilities and tools
3.  **How it should behave** - Personality and constraints
4.  **What it knows** - Domain knowledge and context

### System Context vs. Retrieved Context

| System Context | Retrieved Context |
| :--- | :--- |
| **Static** - Same for every turn | **Dynamic** - Changes per query |
| **Role & behavior** | **Specific facts** |
| **Always included** | **Conditionally included** |
| **Examples:** Agent role, capabilities, guidelines | **Examples:** Course details, user preferences, memories |

# Setup and Initialization

First, let's set up our environment. We need to import the agent code from the `progressive_agents` directory.

In [ ]:
import sys
import os
import asyncio
from pathlib import Path
from dotenv import load_dotenv

# 1. Configure Paths
# We point to 'stage1_baseline_rag' to use the baseline agent
project_root = Path("../../").resolve()
stage1_path = project_root / "progressive_agents" / "stage1_baseline_rag"
sys.path.append(str(stage1_path))

# Add src to path to import models
src_path = project_root / "src"
sys.path.append(str(src_path))

# 2. Load Environment Variables
load_dotenv(project_root / ".env")

print(f"Project Root: {project_root}")
print(f"Agent Path Added: {stage1_path}")

In [ ]:
from agent import setup_agent
from agent.nodes import set_system_instructions

print("Initializing Stage 1 Baseline Agent...")
# We use the Stage 1 baseline agent, but we will override its system prompt
workflow, course_manager = setup_agent(auto_load_courses=True)
print("Agent is ready!")

### Initialize the Agent
We will use the `setup_agent` helper. This function performs a crucial step:
*   It connects to your Redis instance.
*   It checks if the course data exists.
*   If not, it generates 50 sample courses and loads them into Redis.

*Note: This might take a few seconds the first time you run it.*

## Hands-on: Building System Instructions

Let's build system instructions for our Redis University Class Agent step by step. We will override the agent's default system prompt and see how it changes the response.

### Example 1: Minimal System Instructions

Let's start with the bare minimum and see what happens.

In [ ]:
# Minimal system prompt
minimal_prompt = "You are a helpful assistant."

# Override the agent's system instructions
set_system_instructions(minimal_prompt)

# Test it
print("Running agent with minimal instructions...")
result = await workflow.ainvoke({"query": "I need help planning my classes for next semester."})

print("Response with minimal instructions:")
print(result['final_answer'])
print("\n" + "="*80 + "\n")

**Problem:** The agent doesn't know it's a class scheduling agent. It might give generic advice instead of using our course catalog and tools.

### Example 2: Adding Role and Purpose

In [ ]:
# Add role and purpose
role_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations
"""

# Override agent instructions
set_system_instructions(role_prompt)

# Test it
print("Running agent with role instructions...")
result = await workflow.ainvoke({"query": "I need help planning my classes for next semester."})

print("Response with role and purpose:")
print(result['final_answer'])
print("\n" + "="*80 + "\n")

**Better!** The agent now understands its role, but it still doesn't know about our tools or how to behave.

### Example 3: Adding Behavioral Guidelines

In [ ]:
# Add behavioral guidelines
behavior_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when needed
- Provide specific course recommendations with details
- Explain prerequisites and requirements clearly
- Stay focused on course planning and scheduling
- If asked about topics outside your domain, politely redirect to course planning
"""

# Override agent instructions
set_system_instructions(behavior_prompt)

# Test with an off-topic question
print("Running agent with behavior instructions (off-topic query)...")
result = await workflow.ainvoke({"query": "What's the weather like today?"})

print("Response to off-topic question:")
print(result['final_answer'])
print("\n" + "="*80 + "\n")

**Great!** The agent now stays focused on its purpose and redirects off-topic questions.

### Example 4: Complete System Instructions

Let's build the complete system instructions for our agent.

In [ ]:
# Complete system instructions
complete_prompt = """You are the Redis University Class Agent, powered by Redis and the Agent Memory Server.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule for upcoming semesters
- Check prerequisites and course eligibility
- Get personalized course recommendations based on their goals

You have access to:
- A complete course catalog with descriptions, prerequisites, and schedules
- Student preferences and goals (stored in long-term memory)
- Conversation history (stored in working memory)
- Tools to search courses and check prerequisites

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when you need more information
- Provide specific course recommendations with course codes and details
- Explain prerequisites and requirements clearly
- Remember student preferences and reference them in future conversations
- Stay focused on course planning and scheduling
- If asked about topics outside your domain, politely redirect to course planning

Example interactions:
- Student: "I'm interested in machine learning"
  You: "Great! I can help you find ML courses. What's your current year and have you taken any programming courses?"

- Student: "What are the prerequisites for CS401?"
  You: "Let me check that for you." [Use check_prerequisites tool]
"""

print("Complete system instructions:")
print(complete_prompt)

## Testing: Compare Different Instructions

Let's test how different system instructions affect the agent's behavior.

We will run the same query through the agent multiple times, changing **only** the system instructions each time. This shows how the system prompt acts as a "lens" through which the agent interprets the user's request and the retrieved context.

In [ ]:
# Test query
test_query = "I want to learn about databases but I'm not sure where to start."

# Test with different prompts
prompts = {
    "Minimal": minimal_prompt,
    "With Role": role_prompt,
    "With Behavior": behavior_prompt,
    "Complete": complete_prompt
}

for name, prompt in prompts.items():
    # Update agent
    set_system_instructions(prompt)
    
    # Run agent
    print(f"Running with {name} instructions...")
    result = await workflow.ainvoke({"query": test_query})
    
    print(f"\n{'='*80}")
    print(f"{name} Instructions:")
    print(f"{'='*80}")
    print(result['final_answer'])
    print()

## Conclusion

We have successfully crafted effective system instructions and seen how they shape our agent's behavior.

**Key Takeaways:**

1.  **Identity & Role**: Define who the agent is and its domain.
2.  **Capabilities**: List what tools and data it can access.
3.  **Behavioral Guidelines**: Specify tone, interaction style, and edge case handling.
4.  **Constraints**: Explicitly state what the agent should NOT do.

### Next Lesson: Baseline RAG
In Stage 1, we will take this agent and focus on the **retrieval** side. We'll see what happens when we feed it raw, uncurated data from our database.

See you in the next notebook!